In [2]:
import requests

url = "https://www.ah.nl/"

response = requests.get(url)

if response.status_code == 200:
    print("Here is the result:")
    print(f"{response.text[:300]} ...")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


Failed to retrieve the webpage. Status code: 403


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

def find_ah_product_page(search_term):

    options = Options()
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
    options.add_argument(f'user-agent={user_agent}')

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get("https://www.google.com")

    button = driver.find_element('id', 'W0wltc')
    button.click()

    search_bar = driver.find_element("name", "q")
    search_bar.send_keys(search_term)

    search_bar.send_keys(Keys.RETURN)

    time.sleep(3)

    first_result = driver.find_element(By.CSS_SELECTOR, '#rso > div.hlcw0c > div > div > div > div > div > div > div > div.yuRUbf > div > span > a > h3')
    first_result.click()

    button = driver.find_element('id', 'decline-cookies')
    button.click()

    base_url = driver.current_url
    producten_url = base_url + "producten"
    driver.get(producten_url)

    categories = driver.find_elements(By.CSS_SELECTOR, 'div.product-category-overview_category__vqzcb:nth-child(1) > div:nth-child(1) > a:nth-child(1)')
    for category in categories:
        print(category.text)

    driver.quit()

search_term = "Albert Heijn"
find_ah_product_page(search_term)
    


AttributeError: 'WebDriver' object has no attribute 'find_eements'

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys



options = Options()
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.ah.nl/producten")

button = driver.find_element('id', 'decline-cookies')
button.click()

time.sleep(3)





In [3]:
category_urls = []

try:
    category_elements = driver.find_elements(By.CSS_SELECTOR, '.taxonomy-card_title__vBWel a')
    for element in category_elements:
        print(element.text)
        href = element.get_attribute('href')
        category_urls.append(href)
        print(href)
except Exception as e:
    print("Error finding categories:", e)

Aardappel, groente, fruit
https://www.ah.nl/producten/aardappel-groente-fruit
Salades, pizza, maaltijden
https://www.ah.nl/producten/salades-pizza-maaltijden
Vlees, kip, vis, vega
https://www.ah.nl/producten/vlees-kip-vis-vega
Kaas, vleeswaren, tapas
https://www.ah.nl/producten/kaas-vleeswaren-tapas
Zuivel, plantaardig en eieren
https://www.ah.nl/producten/zuivel-plantaardig-en-eieren
Bakkerij en banket
https://www.ah.nl/producten/bakkerij-en-banket
Ontbijtgranen en beleg
https://www.ah.nl/producten/ontbijtgranen-en-beleg
Snoep, koek, chips en chocolade
https://www.ah.nl/producten/snoep-koek-chips-en-chocolade
Tussendoortjes
https://www.ah.nl/producten/tussendoortjes
Frisdrank, sappen, koffie, thee
https://www.ah.nl/producten/frisdrank-sappen-koffie-thee
Wijn en bubbels
https://www.ah.nl/producten/wijn-en-bubbels
Bier en aperitieven
https://www.ah.nl/producten/bier-en-aperitieven
Pasta, rijst en wereldkeuken
https://www.ah.nl/producten/pasta-rijst-en-wereldkeuken
Soepen, sauzen, kruide

In [18]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC




producten = []

def extract_ah_products(driver, category_url):

    
    driver.get(category_url)
    driver.maximize_window()


    button = driver.find_element('id', 'decline-cookies')
    button.click()

    wait = WebDriverWait(driver, timeout = 10)
    # wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="Filter op Merk: AH"]')))


    # try:
        
    #     ah_filter = driver.find_element(By.CSS_SELECTOR, '[aria-label="Filter op Merk: AH"]')
    #     ah_filter.click()
        
    # except NoSuchElementException:
    #     print("AH filter not found")
    


    while True:
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testhook="load-more"]')))
            meer_resulten = driver.find_element(By.CSS_SELECTOR, '[data-testhook="load-more"]')
            meer_resulten.click()

        except:
            break

    products = driver.find_elements(By.CSS_SELECTOR, '[data-testhook="product-card"]')
    for product in products:
        try:
            alleen_in_de_winkel = product.find_element(By.CSS_SELECTOR, '[data-testhook="product-availability-label"]')
            if "Alleen in de winkel" in alleen_in_de_winkel.text:
                product_name = product.find_element(By.CSS_SELECTOR, '[data-testhook="product-title"]').text
                print(product_name)
                producten.append(product_name)
        except NoSuchElementException:
            continue
    
    driver.quit()


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

for category in category_urls:
    extract_ah_products(driver, category)


KeyboardInterrupt: 

In [34]:
print(producten)

[]


Albert Heijn: boodschappen doen bij de grootste supermarkt. https://www.ah.nl/
Albert Heijn (@albertheijn) · X. https://twitter.com/albertheijn?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor
Albert Heijn (supermarkt). https://nl.wikipedia.org/wiki/Albert_Heijn_(supermarkt)
Albert Heijn. https://www.facebook.com/albertheijn/?locale=nl_NL
Albert Heijn (@albertheijn) • Instagram photos and videos. https://www.instagram.com/albertheijn/
Albert Heijn supermarkt - Apps on Google Play. https://play.google.com/store/apps/details?id=com.icemobile.albertheijn&hl=en_US
Albert Heijn. https://en.wikipedia.org/wiki/Albert_Heijn
Meer resultaten. https://www.google.com/search?q=Albert+Heijn&sca_esv=a9728410f1a0f0fe&source=hp&ei=_GjPZb6tHrC6i-gPmIqn6Ac&iflsig=ANes7DEAAAAAZc93DDicKNT5NBXqfvmZkySf52GsfzT4&uact=5&oq=Albert+Heijn&gs_lp=Egdnd3Mtd2l6IgxBbGJlcnQgSGVpam4yERAuGIAEGLEDGMkDGMcBGNEDMgsQABiABBixAxiSAzILEAAYgAQYigUYkgMyCBAAGIAEGLEDMggQABiABBixAzIIEAAYgAQYsQMyBRAAGIAEMgUQABiABDIFEAAYgAQyCxAuG